In [1]:
import csv
import pickle
import re
from tqdm import tqdm
from collections import Counter
import random
from functools import partial
import math
import time
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import numpy as np

In [2]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
    clean_recipes = data['clean_recipes']
    messy_recipes = data['messy_recipes']
    messy_name = data['messyname']
    messy_id = data['messyid']
    
clean_ingredients = set(['avocados', 'eggplants', 'creme fraiche', 'oreos', 'steaks', 'steak', 'raisin', 'halfandhalf', 'prawn', 'aubergine', 'lychee', 'lobsters'])
for r in clean_recipes:
    clean_ingredients.update(r)
print(len(clean_ingredients))

7115


In [4]:
with open('./ingredients v1.csv', 'r') as f:
    reader = csv.reader(f)  
    rows = [l for l in reader if l[7].lower() == 'ingredients']
    for row in tqdm(rows):
        ingr = row[8]
        ingr = ingr[:-1] if ingr.endswith('.') else ingr
        ingr = re.sub(r'\([^)]*\)', '', ingr).lower()
        ingredients = [z.strip() for z in ingr.split(',') if z.strip().replace(' ', '').isalpha() and len(z.strip()) <= 100]
        new_ingredients = []
        for i in ingredients:
            new_ingredients.extend([o.strip() for o in i.split('and') if o.strip()])
        clean_ingredients.update(new_ingredients)

print(len(clean_ingredients))

100%|██████████| 9940/9940 [00:00<00:00, 18537.23it/s]

15422


In [6]:
def select_n_max_length_ingr(l, n=1):
    l.sort(key=lambda x: -len(x))
    return l[:n]

def select_all(l):
    return l

def select_percentage_and_min(l, percentage=0.65, min_count=2):
    return select_n_max_length_ingr(l, n=max(min_count, round(len(l)*percentage)))

def clean_ingredient_list(l, selection_policy=select_percentage_and_min):
    #print(l)
    #print('callend\n\n\n')
    recipe = set()
    
    for ingredient in l:
        k = []
        for ci in clean_ingredients:
            if ' '+ci.strip()+' ' in ' '+ingredient+' ':
                k.append(ci)
            
        recipe.update(selection_policy(k))
        #if len(k) == 2:
        #    print(ingredient, '-->', k)
        
    return recipe

In [8]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
all_recipes_raw = messy_recipes

In [11]:
chunked_list = list(chunks(all_recipes_raw, 400))

In [12]:
chunked_list[0]

[('allpurpose flour',
  'gruyere cheese grated',
  'milk',
  'semisoft cheese page grated',
  'penne',
  'stick unsalted butter',
  'semihard cheese page grated about',
  'to chipotle chili powder',
  'cheddar grated',
  'makes about',
  'beechers flagship cheese sauce recipe follows',
  'garlic powder',
  'to chipotle chili powder see note',
  'kosher salt'),
 ('sliced celery',
  'vinegar',
  'minced green pepper',
  'mayonnaise possibly salad dressing',
  'cubed american cheese',
  'dry dill weed',
  'salt',
  'elbow macaroni',
  'minced pimento'),
 ('green bell pepper cut into dice',
  'extravirgin olive oil for drizzling',
  'red bell pepper cut into dice',
  'tomatoes quartered',
  'cucumber cut into dice',
  'yellow bell pepper cut into dice',
  'leaves basil finely',
  'kosher salt',
  'red onion cut into dice'),
 ('kernel corn drained',
  'milk',
  'frenchfried onions',
  'mashed potatoes box homestyle',
  'cheddar cheese',
  'butter',
  'water'),
 ('graham cracker crust',
  'p

In [13]:
find_n_ingredients = 6000
select_top_n_ingredients = 3500
ingredient_preselection = list()

p = Pool(16)

for r_chunk in tqdm(chunked_list):
    r_chunk_new = p.map(clean_ingredient_list, r_chunk)
    for r_new in r_chunk_new:
        ingredient_preselection.extend(r_new)
        
    unique_ingredient_count = len(set(ingredient_preselection))
    if unique_ingredient_count > find_n_ingredients:
        break
    print(f'Found {unique_ingredient_count} unique ingredients') 
        
c = Counter(ingredient_preselection)
clean_ingredients = list([i for i, n in c.most_common()][:select_top_n_ingredients])
assert len(clean_ingredients) == select_top_n_ingredients

  0%|          | 1/2575 [00:19<13:48:53, 19.32s/it]

Found 1302 unique ingredients


  0%|          | 2/2575 [00:36<13:24:07, 18.75s/it]

Found 1855 unique ingredients


  0%|          | 3/2575 [00:54<13:09:26, 18.42s/it]

...

Found 5994 unique ingredients


  4%|▍         | 111/2575 [1:29:57<20:56:17, 30.59s/it]

Found 6000 unique ingredients


Process ForkPoolWorker-13:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-10:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-12:
Process ForkPoolWorker-9:
Process ForkPoolWorker-14:
Process ForkPoolWorker-3:
Process ForkPoolWorker-11:
Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", 

  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/F970614/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwa

In [32]:
select_top_n_ingredients = 1000
clean_ingredients = list([i for i, n in c.most_common()][:select_top_n_ingredients])
assert len(clean_ingredients) == select_top_n_ingredients

In [33]:
print(f'Selected {len(clean_ingredients)} ingredients:')
for f in list(c.most_common())[:select_top_n_ingredients]:
    print(f)

Selected 1000 ingredients:
('salt', 18521)
('pepper', 16050)
('butter', 12560)
('garlic', 12316)
('sugar', 11223)
('flour', 9456)
('onion', 9432)
('water', 8648)
('olive oil', 8626)
('olive', 8497)
('sliced', 7053)
('eggs', 7026)
('milk', 6814)
('black pepper', 6418)
('cheese', 5958)
('powder', 5846)
('vanilla', 5694)
('cream', 5469)
('lemon', 5436)
('chicken', 5366)
('sauce', 5300)
('tomatoes', 4943)
('egg', 4579)
('ground', 4501)
('white', 4314)
('brown sugar', 4124)
('vinegar', 4090)
('onions', 4070)
('vegetable', 3930)
('lemon juice', 3927)
('parsley', 3874)
('cinnamon', 3865)
('brown', 3849)
('ground black pepper', 3604)
('garlic cloves', 3524)
('vegetable oil', 3055)
('baking powder', 3011)
('vanilla extract', 2941)
('soda', 2815)
('tomato', 2810)
('celery', 2759)
('unsalted butter', 2755)
('ginger', 2636)
('baking soda', 2632)
('mustard', 2618)
('chocolate', 2478)
('cream cheese', 2464)
('parmesan', 2449)
('leaves', 2435)
('sour cream', 2432)
('potatoes', 2425)
('cheddar', 2350)

In [34]:
recipes = []

p = Pool(16)

for r_chunk in tqdm(chunked_list):
    r_chunk_new = p.map(clean_ingredient_list, r_chunk)
    for r_new in r_chunk_new:
        recipes.append(tuple(sorted(r_new)))
    print(f'{len(recipes)} processed recipes')



  0%|          | 0/2575 [00:00<?, ?it/s]

  0%|          | 1/2575 [00:01<48:26,  1.13s/it]

400 processed recipes




  0%|          | 2/2575 [00:02<48:51,  1.14s/it]

800 processed recipes




  0%|          | 3/2575 [00:03<47:44,  1.11s/it]

...

1029200 processed recipes




100%|█████████▉| 2574/2575 [2:03:42<00:01,  1.32s/it]

1029600 processed recipes




100%|██████████| 2575/2575 [2:03:42<00:00,  2.88s/it]

1029720 processed recipes


In [55]:
recipes[0]

('butter',
 'cheddar',
 'cheese',
 'chili powder',
 'chipotle',
 'flour',
 'garlic',
 'garlic powder',
 'gruyere cheese',
 'kosher',
 'kosher salt',
 'milk',
 'penne',
 'powder',
 'sauce',
 'unsalted butter')

In [72]:
indexed_recipes = []
recipes_names = []
for index, r in enumerate(recipes):
    if r and messy_name[index]!='':
        recipes_names.append(messy_name[index])
        indexed_recipes.append(np.array([clean_ingredients.index(i) for i in r]))

In [73]:
len(indexed_recipes)

1029100

In [78]:
len(recipes_names)

1029100

In [85]:
for i in recipes_names[1029035:]:
    print(i)

Can't Wait Tailgate BBQ Tacos Recipe alender
Cranberry and Apricot Chutney
Pickled Jalapenos Recipe
Lemon Zest Muffins (Gluten Free)
Chicken Sausage, Pepper and Onion Subs
Pasta with Chianti and Tomatoes
Cheesy Spinach Dip
Five Star Chile Rellenos Casserole
Boston Banana Brown Bread
Jumbo Pecan Date Oatmeal Cookies
Warm Apple and Goat Cheese Salad
Chicken Breast + Shio-Koji + Olive Oil Cooked in the Microwave
Homemade Naan with Garlic and Cilantro
Lamb-and-Apricot Tagine with Almond Couscous
Pork Chop 'n' Rice Casserole
Potato Sauce
Marie's Christmas Cake
Italian Sausage and Peppers (Low-Carb, Slow Cooker Crock Pot)
Bread And Cheese Casserole Recipe
Potato Latkes with Apple Cranberry Sauce
Crispy-Crusted Apple Cake
Corned Beef and Barley Soup
Eggplant (Aubergine) in Hot Sauce
Modern Boeuf a La Mode
Gabbie's Sweet Potato Pie
Honey Butter Split Top Whole Wheat Bread Recipe
8-Minute Hard-Boiled Eggs
Ham Florentine Mini Cups
Armadillo Eggs
Heirloom Tomato Gazpacho
Almond Bark Candy
Breakfa

### Save

In [45]:
f = open('cleanrecipe.txt', 'w')
for index, recipe in enumerate(indexed_recipes):
    for material in recipe:
        f.write(str(index+1)+','+str(material+1)+'\n') 
f.close()

In [80]:
with open('name.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(recipes_names))

In [88]:
with open('ingredient.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(clean_ingredients))

<function TextIOWrapper.close()>

In [52]:
len(indexed_recipes)

1029100

In [53]:
len(recipes)

1029720

In [56]:
len(messy_name)

1029720